### References

* [Observations of Large-Scale Anisotropy in the Arrival Directions of Cosmic Rays Above...](https://arxiv.org/abs/1709.07321)
* [Healpy](https://arxiv.org/abs/1709.07321)
* [pandas](https://pandas.pydata.org/)
* [numpy](https://numpy.org/)
* [scipy](https://scipy.org/)
* [matplotlib](https://matplotlib.org/)

In [ ]:
%matplotlib inline

from IPython.display import HTML, Latex, Math, display

# Data analysis
import numpy as np
import pandas as pd

# Create spherical-map
import healpy as hp

# Manage & convert astro coordinates
import astropy.units as u
import astropy.coordinates as Coord

# Tools for Rayleigh analysis
from scipy.optimize import minimize, least_squares, curve_fit
from scipy.stats import chi2
from scipy.special import erfcinv

# Plotting
import matplotlib
from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
def ReadData (fin):
    """
    Lee el fichero que se le pasa por argumento y devuelve un pandas.DataFrame
    """
    return pd.read_csv(fin)


def ShowStoredDataFrame (df):
    display(HTML(df.to_html()))
    
    
def RejectParameters (df):
    return df[['id','sd_energy','sd_phi','sd_dphi',
                 'sd_theta','sd_dtheta','sd_exposure',
                 'sd_l','sd_b','sd_ra','sd_dec',
                'fd_calEnergy','fd_dcalEnergy']]
                 
    
    
def MapToHealpyCoord(galCoord, l, b):
    
    theta = np.pi/2 - b
    phi = l

    if(not galCoord): # If equatorial coordinates
        phi -= np.pi
    return phi, theta

def HealpyCoordToMap(galCoord, phi, theta):
    
    b = np.pi/2 - theta
    l = phi
    if(not galCoord): # If equatorial coordinates: l+π, projected to [0, 2*π]
        l = np.where(l < np.pi, l + np.pi, l - np.pi) 
    return l, b


def LoadShapedData(galCoord = False, Emin = 8):
    
    # Coordinate conversion, returns (phi, theta)
    coords = MapToHealpyCoord(galCoord,
                              np.radians(data.sd_l if galCoord else data.sd_ra),
                              np.radians(data.sd_b if galCoord else data.sd_dec))

    # Event selection above Emin
    unmasked = (data.sd_energy>Emin)
    dataset = pd.DataFrame(dict(zip(['phi', 'theta'], coords)))[unmasked]
    return dataset





def LoadCountMap(dataset, nside):
    
    # Pixel index for each events of coordinates (theta, phi)
    index = hp.ang2pix(nside, dataset["theta"].values, dataset['phi'].values)

    # Count map of parameter nside
    npix = hp.nside2npix(nside)# npix corresponds to the number of pixel associated to a NSIDE healpy map
    count_map = np.histogram(index, bins=np.arange(npix + 1))[0]
    
    return count_map

def PlotHPmap(HPmap, nside, galCoord, title, color_bar_title, projection="hammer", cmap='afmhot', vmin=None, vmax=None, fig=None, ax=None):
    
    # Transform healpix map into matplotlib map (grid_map)
    xsize = 2000 # grid size for matplotlib
    ysize = int(xsize/2.)
    theta = np.linspace(np.pi, 0, ysize)
    phi   = np.linspace(-np.pi, np.pi, xsize)
    PHI, THETA = np.meshgrid(phi, theta)
    grid_pix = hp.ang2pix(nside, THETA, PHI)
    grid_map = HPmap[grid_pix]

    # Define the figure
    width = 15# width of the figure
    if fig is None:
        doubleFig = False
        fig = plt.figure(figsize=(width,width/1.5))
        fig.suptitle(title, size=30)
    else:
        doubleFig = True
    if ax is None:
        ax = fig.add_subplot(111, projection=projection)
    labelsize = 16
    ax.tick_params(axis='x', labelsize=labelsize)
    ax.tick_params(axis='y', labelsize=labelsize)
            
    # Set the size of the other fonts
    fontsize = 22
    font = {'size': fontsize}
    matplotlib.rc('font', **font)
    
    # minimum and maximum values along the z-scale (colorbar)
    if vmax is None:
        vmax = np.max(HPmap)
    if vmin is None:
        vmin = np.min(HPmap)   
    
    # Plot the map, reverse the longitude axis "[::-1]" (astronomical convention)
    longitude = np.radians(np.linspace(-180, 180, xsize))
    latitude = np.radians(np.linspace(-90, 90, ysize))
    image = ax.pcolormesh(longitude[::-1], latitude, grid_map, rasterized=True, 
                          cmap=plt.get_cmap(cmap), shading='auto', vmin=vmin, vmax=vmax)
    if not doubleFig:
        cb = fig.colorbar(image, orientation='horizontal', shrink=.6, pad=0.05)
        cb.set_label(color_bar_title, size=fontsize)
    
    # Plot the labels considering if it is galactic or equatorial coordinates
    if(galCoord):
        ax.set_xticklabels([r"150$\degree$", r"120$\degree$", r"90$\degree$", r"60$\degree$", r"30$\degree$", r"GC", r"330$\degree$", r"300$\degree$", r"270$\degree$", r"240$\degree$", r"210$\degree$"])
        ax.set_title("Galactic")
        ax.set_xlabel('l', size=labelsize)
        ax.set_ylabel('b', size=labelsize)
    else:
        ax.set_xticklabels([r"330$\degree$", r"300$\degree$", r"270$\degree$", r"240$\degree$", r"210$\degree$", r"180$\degree$", r"150$\degree$", r"120$\degree$", r"90$\degree$", r"60$\degree$", r"30$\degree$"])
        ax.set_title("Equatorial")
        ax.set_xlabel('R.A.', size=labelsize)
        ax.set_ylabel('Dec.', size=labelsize)
        
    ax.grid(True, alpha=0.25)

    if doubleFig:
        return image


    
    
def exposure(dec):
    
    theta_max = np.radians(60) # Maximum zenith angle in the dataset
    l = np.radians(-35.23) # Latitude of the center of the array (near Malargüe - Argentina)
    
    arg = (np.cos(theta_max) - np.sin(l)*np.sin(dec)) / (np.cos(l)*np.cos(dec))
    hm = np.arccos(arg.clip(-1, 1))

    return np.cos(l)*np.cos(dec)*np.sin(hm) + hm*np.sin(l)*np.sin(dec)

def Get_ra_dec(nside, galCoord):
    
     # Get the healpy coordinates of each pixel 
    npix = hp.nside2npix(nside)
    phi, theta = hp.pix2ang(nside, np.arange(npix))

    # Transform the (y,x) values into whether galactic coordinates or whether equatorial coordinates.
    if(galCoord): #If galactic coordinates, get the declination value for a given (l,b)
        l, b = HealpyCoordToMap(True, theta, phi)
        c = Coord.SkyCoord(l=l*u.radian, b=b*u.radian, frame='galactic')
        ra = c.icrs.ra.radian
        dec = c.icrs.dec.radian
    else:
        ra, dec  = HealpyCoordToMap(False, theta, phi)
        
    return ra, dec

def LoadExposureMap(totalExposure, nside, galCoord):
   
    ra, dec = Get_ra_dec(nside, galCoord)
    
    # Compute the exposure_map for each pixel regarding its declination.
    exposure_map = exposure(dec)*totalExposure/np.sum(exposure(dec))
    
    return exposure_map


## Parameters

In [ ]:
ENERGY_MIN_ARRIVAL = # Emin
GAL_COORD = True
NSIDE = 64
RADIOUS_DEG = # top-hat radius in degrees

# Data 

In [ ]:
raw_data = ReadData("summary/dataSummary.csv")
raw_data = RejectParameters(raw_data)
e_cut = raw_data["sd_energy"] > ENERGY_MIN_ARRIVAL

data = raw_data[e_cut]
total_exposure = data["sd_exposure"].iloc[-1]

In [ ]:
cols = raw_data.columns
print("Datos:")
for c in cols:
    print("\t",c)

## Energy distributions

In [ ]:
plt.figure(figsize=(6.2,6))
plt.semilogy()
b = np.linspace(0,150,50)
plt.hist(data['sd_energy'],bins=b,alpha=0.5,label=r"$E_{SD}$");
plt.hist(data[''],bins=b,alpha=0.5,label="$E_{FD}$"); # introduce el nombre de la variable para pintar la energía FD
leg = plt.legend()
leg.draw_frame(False);
plt.tight_layout()
plt.xlabel(r"$E [eV]$",size=15);
plt.ylabel(r"$Counts$",size=15);

## Angular distributions

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(121)
b = np.linspace(0,60,10)
plt.hist(data[''],bins=b); # representa el ángulo acimutal
plt.ylabel(r"$Counts$",size=15)
plt.xlabel(r"$\theta$",size=15)
plt.subplot(122)
b = np.linspace(0,150,20)
plt.hist(data[''],bins=b); # representa el ángulo cenital
plt.tight_layout()
plt.xlabel(r"$\phi$",size=15)

# Data selection

In [ ]:
data_arrival = LoadShapedData(GAL_COORD,ENERGY_MIN_ARRIVAL)

In [ ]:
display(data_arrival)

In [ ]:
count_map = LoadCountMap(data_arrival, NSIDE)
title=f"Count Map, $E > {ENERGY_MIN_ARRIVAL}$ EeV"
color_bar_title = "# events per pixel"
PlotHPmap(count_map, NSIDE, GAL_COORD, title, color_bar_title)

In [ ]:
title = "Exposure Map"
color_bar_title = r"Exposure [$\rm km^{2} \, sr \, yr$ / pixel]"


# Plot the two exposure map
rcParams['figure.figsize'] = 20, 14 #Size of figs
fig, ax = plt.subplots(1, 2, subplot_kw={'projection': 'hammer'})
fig.suptitle(title, fontsize=30)

for galCoord in [0, 1]:
    exposure_map = LoadExposureMap(total_exposure, NSIDE, galCoord)
    image = PlotHPmap(exposure_map, NSIDE, galCoord, title, color_bar_title, ax=ax[galCoord], fig=fig)




fig.subplots_adjust(top=1.4)
cbar_ax = fig.add_axes([0.25, 0.55, 0.5, 0.03])
cb = fig.colorbar(image, orientation='horizontal', shrink=.6, pad=0.05, cax=cbar_ax)
cb.set_label(color_bar_title, size=22)


plt.show()

In [ ]:
def top_hat_beam(radius, nside):
    
    b = np.linspace(0.0, np.pi, 10000)
    bw = np.where(abs(b) <= radius, 1, 0)
    return hp.sphtfunc.beam2bl(bw, b, lmax=nside*3) #beam in the spherical harmonics space

def LoadSmoothedMap(hp_map, radius_deg, nside):
    
    radius = np.radians(radius_deg)
    solid_angle = 2.*np.pi*(1. - np.cos(radius))
    return hp.smoothing(hp_map, beam_window=top_hat_beam(radius, nside), verbose=False) / solid_angle

# Define title and the color bar title of the smoothed exposure map
title = f"Smoothed Exposure Map, $R={RADIOUS_DEG}\\degree$"
color_bar_title = r"Exposure [$\rm km^{2} \, sr \, yr$ / pixel]"

# Plot the two exposure map
rcParams['figure.figsize'] = 20, 14 #Size of figs
fig, ax = plt.subplots(1, 2, subplot_kw={'projection': 'hammer'})
fig.suptitle(title, fontsize=30)

for galCoord in [0, 1]:
    exposure_map = LoadExposureMap(total_exposure, NSIDE, galCoord)
    exposure_map_smoothed = LoadSmoothedMap(exposure_map, RADIOUS_DEG, NSIDE)
    image = PlotHPmap(exposure_map_smoothed, NSIDE, galCoord, title, color_bar_title, ax=ax[galCoord], fig=fig)



fig.subplots_adjust(top=1.4)
cbar_ax = fig.add_axes([0.25, 0.55, 0.5, 0.03])
cb = fig.colorbar(image, orientation='horizontal', shrink=.6, pad=0.05, cax=cbar_ax)
cb.set_label(color_bar_title, size=22)


# Define title and the color bar title of the smoothed count map
title = f"Smoothed Count Map, $E >{ENERGY_MIN_ARRIVAL}$ EeV, $R = {RADIOUS_DEG}\\degree$"
color_bar_title = r"# events per pixel"

# Plot the two exposure map
rcParams['figure.figsize'] = 20, 14 #Size of figs
fig, ax = plt.subplots(1, 2, subplot_kw={'projection': 'hammer'})
fig.suptitle(title, fontsize=30)

for galCoord in [0, 1]:
    
    dataset = LoadShapedData(galCoord, ENERGY_MIN_ARRIVAL)# data above 8 EeV
    count_map = LoadCountMap(dataset, NSIDE)
    count_map_smoothed = LoadSmoothedMap(count_map, RADIOUS_DEG, NSIDE)
    image = PlotHPmap(count_map_smoothed, NSIDE, galCoord, title, color_bar_title, ax=ax[galCoord], fig=fig)

fig.subplots_adjust(top=1.4)
cbar_ax = fig.add_axes([0.25, 0.55, 0.5, 0.03])
cb = fig.colorbar(image, orientation='horizontal', shrink=.6, pad=0.05, cax=cbar_ax)
cb.set_label(color_bar_title, size=22)

plt.show()


In [ ]:
def RA_Binning(ra, nbins, normalize=True):
    
    # Get the average flux, the error by binning in r.a. the flux_map
    n_bin, bin_edges = np.histogram(ra, bins=nbins)
    err_n_bin = np.sqrt(n_bin)
    ra_bin = 0.5 * (bin_edges[:-1] + bin_edges[1:])

    # Normalize to average value
    if(normalize):
        weights = 1./(err_n_bin*err_n_bin)
        average = np.sum(weights*n_bin)/np.sum(weights)
        n_bin = n_bin / average
        err_n_bin = err_n_bin / average
        
    return ra_bin, n_bin, err_n_bin

def FuncConst(x, a):
    return a*np.ones_like(x)

def FuncCos(x, a, b, c):
    return (a + b*np.cos(np.radians(x)-c))

def Chi2_Const(par, args):
    x, res, err = args[0], args[1], args[2]
    return np.sum((res - FuncConst(x, par[0]))**2 / err**2)

def TwoSidedSignificance(prob):
    return np.sqrt(2.)*erfcinv(prob)


def FitConst (ra_bin,n_bin,err_n_bin):
    result_Const, var_Const = curve_fit(FuncConst, ra_bin, n_bin, sigma = err_n_bin, maxfev = 10000)
    return result_Const

def Rayleigh(ra):
    a = 2. * np.sum(np.cos(np.radians(ra))) / ra.size
    b = 2. * np.sum(np.sin(np.radians(ra))) / ra.size
    alpha0 = np.arctan2(b,a)
    r = np.sqrt(a*a + b*b)
    
    p = np.exp(-ra.size*r*r/4.)
    sigma = TwoSidedSignificance(p)
    
    err_r = np.sqrt(2./ra.size)
    err_alpha0 = np.sqrt(2./ra.size) / r
    
    display(Latex(f'''With Rayleigh statistics, the first harmonic is preferred at {sigma:2.1f} sigma compared to isotropy, with parameters:
    \\begin{{align}}
    r &= {r:5.2} \\pm {err_r:5.2}\\\\
    \\alpha_0 &= {np.degrees(alpha0):4.0f}^\\circ \pm {np.degrees(err_alpha0):4.0f}\\\\
    \\end{{align}}
    '''))
    return r, alpha0

In [ ]:
# Vuelve a seleccionar los datos que se usan en el análisis
ra_sel = raw_data["sd_ra"][raw_data["sd_energy"]>ENERGY_MIN_ARRIVAL]

# Transforma la ascensión recta de -180,180 a 0,360                          
ra_sel[ra_sel < 0] += 360. # change [-180° , 180°] to [0° , 360°]

nbins = 12 #Number of bin used


# Rayleigh analysis
r, alpha0 = Rayleigh(ra_sel)

ra_bin, n_bin, err_n_bin = RA_Binning(ra_sel, nbins)
# Mira los argumentos que necesita la función FitConst y pásalos
resConst = FitConst(ra_bin,n_bin,err_n_bin)

## Results

In [ ]:
# Create the figure
width = 15# width of the figure
fig = plt.figure(figsize=(width, width/1.5))
ax = fig.add_subplot(111)
ax.set_xlabel("Right Ascension [deg]")
ax.set_ylabel("Normalized rate")
plt.xticks(np.arange(0, 360 + 1, 60))
ax.invert_xaxis()
plt.xlim(360, 0.)
    
# Plot the data
xplot = np.linspace(0, 360, 100)
 

ax.plot(xplot, FuncCos(xplot, 1, r, alpha0), label="Rayleigh analysis")
ax.plot(xplot, FuncConst(xplot, resConst), label="Constant fit (isotropy)")
ax.errorbar(ra_bin, n_bin, yerr=err_n_bin, fmt='o', label="Data E > "+str(ENERGY_MIN_ARRIVAL)+" EeV",
           c="black")
plt.legend(loc="lower right")
plt.show()